# Notebook 01 — Data Loading & Cleaning

## 01.1 Imports and Reproducibility

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

RANDOM_STATE = 42
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 140)


## 01.2 Load Dataset

In [2]:
DATA_PATH = r"C:\Users\farbo\OneDrive\Desktop\churn-analysis\data\telco_churn.csv"

if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"File not found at: {DATA_PATH}")

df = pd.read_csv(DATA_PATH)
print("Loaded:", DATA_PATH)
print("Shape:", df.shape)
df.head()


Loaded: C:\Users\farbo\OneDrive\Desktop\churn-analysis\data\telco_churn.csv
Shape: (7043, 21)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


## 01.3 Standardize column names

In [3]:
def clean_column_names(cols):
    return (
        cols.str.strip()
            .str.replace(" ", "_", regex=False)
            .str.replace("-", "_", regex=False)
            .str.lower()
    )

df.columns = clean_column_names(df.columns)
print("Columns:\n", df.columns.tolist())


Columns:
 ['customerid', 'gender', 'seniorcitizen', 'partner', 'dependents', 'tenure', 'phoneservice', 'multiplelines', 'internetservice', 'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport', 'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling', 'paymentmethod', 'monthlycharges', 'totalcharges', 'churn']


## 01.4 Basic data audit (types, duplicates, missing)

In [4]:
print("\n--- Info ---")
display(df.info())

print("\n--- Duplicate rows ---")
print(df.duplicated().sum())

print("\n--- Missing values per column (raw) ---")
missing_raw = df.isna().sum().sort_values(ascending=False)
print(missing_raw[missing_raw > 0])



--- Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerid        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   seniorcitizen     7043 non-null   int64  
 3   partner           7043 non-null   object 
 4   dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   phoneservice      7043 non-null   object 
 7   multiplelines     7043 non-null   object 
 8   internetservice   7043 non-null   object 
 9   onlinesecurity    7043 non-null   object 
 10  onlinebackup      7043 non-null   object 
 11  deviceprotection  7043 non-null   object 
 12  techsupport       7043 non-null   object 
 13  streamingtv       7043 non-null   object 
 14  streamingmovies   7043 non-null   object 
 15  contract          7043 non-null   object 
 16  paperlessbilling  7043 non-n

None


--- Duplicate rows ---
0

--- Missing values per column (raw) ---
Series([], dtype: int64)


## 01.5 Convert blank strings to NaN

In [5]:
# Convert empty/whitespace-only strings to NaN for object columns
obj_cols = df.select_dtypes(include=["object"]).columns
df[obj_cols] = df[obj_cols].replace(r"^\s*$", np.nan, regex=True)

print("\n--- Missing values per column (after blank->NaN) ---")
missing = df.isna().sum().sort_values(ascending=False)
print(missing[missing > 0])



--- Missing values per column (after blank->NaN) ---
totalcharges    11
dtype: int64


## 01.6 Fix totalcharges data type (object → numeric)

In [6]:
if "totalcharges" in df.columns:
    df["totalcharges"] = pd.to_numeric(df["totalcharges"], errors="coerce")
    print("Converted totalcharges to numeric.")
    print("totalcharges dtype:", df["totalcharges"].dtype)
else:
    print("Note: totalcharges column not found in your dataset version.")


Converted totalcharges to numeric.
totalcharges dtype: float64


## 01.7 Handle missing values (minimal, safe cleaning)

In [7]:
# Minimal missing handling
if "totalcharges" in df.columns and "tenure" in df.columns:
    missing_tc = df["totalcharges"].isna().sum()
    print("Missing totalcharges:", missing_tc)

    # rule: new customers (tenure==0) => totalcharges=0
    mask_new = (df["tenure"] == 0) & (df["totalcharges"].isna())
    df.loc[mask_new, "totalcharges"] = 0

    # remaining missing -> median
    if df["totalcharges"].isna().any():
        median_tc = df["totalcharges"].median()
        df["totalcharges"] = df["totalcharges"].fillna(median_tc)
        print("Filled remaining totalcharges with median:", median_tc)

print("\n--- Missing values per column (final) ---")
missing_final = df.isna().sum().sort_values(ascending=False)
print(missing_final[missing_final > 0])


Missing totalcharges: 11

--- Missing values per column (final) ---
Series([], dtype: int64)


## 01.8 Drop obvious non-feature ID column (keep separately)
Customer IDs are typically unique identifiers and not useful predictive features.

In [8]:
ID_COL = None
for candidate in ["customerid", "customer_id", "id"]:
    if candidate in df.columns:
        ID_COL = candidate
        break

if ID_COL:
    print("Identified ID column:", ID_COL)
else:
    print("No obvious ID column found.")


Identified ID column: customerid


## 01.9 Define the target (churn) cleanly

We’ll map Yes/No → 1/0 for modeling.

In [9]:
if "churn" not in df.columns:
    raise ValueError("Target column 'churn' not found. Please confirm your CSV has a churn column.")

# Normalize churn labels
df["churn"] = df["churn"].astype(str).str.strip().str.lower()

valid = set(df["churn"].dropna().unique())
print("Unique churn values:", valid)

# Common mappings in this dataset: yes/no
mapping = {"yes": 1, "no": 0}
if not valid.issubset(set(mapping.keys())):
    raise ValueError(f"Unexpected churn labels: {valid}. Expected only {set(mapping.keys())}.")

df["churn"] = df["churn"].map(mapping).astype(int)

churn_rate = df["churn"].mean()
print(f"Churn rate: {churn_rate:.3f} ({churn_rate*100:.1f}%)")

print("\nClass counts:")
print(df["churn"].value_counts())


Unique churn values: {'no', 'yes'}
Churn rate: 0.265 (26.5%)

Class counts:
churn
0    5174
1    1869
Name: count, dtype: int64


## 01.10 Final dataset snapshot

We’ll create:
* df_clean = cleaned dataframe
* X_raw, y = split target vs predictors (raw, not encoded yet)

In [10]:
df_clean = df.copy()

y = df_clean["churn"]
X_raw = df_clean.drop(columns=["churn"])

print("Final df_clean shape:", df_clean.shape)
print("X_raw shape:", X_raw.shape, "| y shape:", y.shape)

df_clean.head()


Final df_clean shape: (7043, 21)
X_raw shape: (7043, 20) | y shape: (7043,)


,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,0
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,0
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,1
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,0
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,1


## 01.11 Save a cleaned copy (optional but useful)

This helps make later notebooks fast/reproducible.

In [11]:
OUT_PATH = r"C:\Users\farbo\OneDrive\Desktop\churn-analysis\data\telco_churn_clean.csv"
df_clean.to_csv(OUT_PATH, index=False)
print("Saved cleaned dataset to:", OUT_PATH)


Saved cleaned dataset to: C:\Users\farbo\OneDrive\Desktop\churn-analysis\data\telco_churn_clean.csv
